In [1]:
%pip install pytorch_metric_learning datasets

In [2]:
import pandas as pd
import json
import torch
from torch.optim import AdamW
from transformers import AutoModel, AutoTokenizer, get_linear_schedule_with_warmup
from datasets import load_dataset
from torch.utils.data import DataLoader
from pytorch_metric_learning import miners, losses
from datasets import load_metric
import time
import torch
import math

In [3]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/MyDrive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
if torch.backends.mps.is_available():
    device = torch.device('mps')
    print("Device: mps")
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("Device: cuda")
else:
    device = torch.device('cpu')
    print("Device: cpu")

Device: cuda


#### Tokenise data

In [ ]:
class CitationDataSet:
    def __init__(self, source, tokenizer_name='allenai/scibert_scivocab_uncased'):
        self.dataset = load_dataset("csv", data_files=source)
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
        self.column_names = self.dataset['train'].column_names
        self.col_name = self.column_names[0]
        self.label_col = self.column_names[-1]

    def tokenize(self, examples, max_length=256):
        id_masks_all_cols = []
        batch_size = len(examples[self.col_name])

        # Tokenize examples for each column (ignore last column ie. 'label')
        for k in self.column_names[:-1]:
            id_mask = self.tokenizer(examples[k], truncation=True, padding='max_length', max_length=max_length)
            id_masks_all_cols.append(id_mask)

        zipped_id_mask = {}
        id_mask_col = id_masks_all_cols[0]

        # Zips all columns together for each feature, input_id/attention_mask
        for feature in id_mask_col:
            zipped_id_mask[feature] = [[id_mask[feature][i] for id_mask in id_masks_all_cols] for i in range(batch_size)]

        zipped_id_mask[self.label_col] = examples[self.label_col]
        return zipped_id_mask


    def get_dataloader(self):
        # Shape = [features, batch_size, (anchor, pos)/label]
        dataset = self.dataset['train'].map(self.tokenize, batched=True, remove_columns=self.column_names)

        dataset.set_format("torch")
        train_dataloader = DataLoader(dataset, shuffle=True, batch_size=8)
        return train_dataloader

#### Fine Tune Model

In [ ]:
# Uses [CLS] token representation
def encoder(batch, model):
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['label']

    # Flatten to encode all at once
    input_ids = torch.cat((input_ids[:, 0], input_ids[:, 1])).to(device)
    attention_mask = torch.cat((attention_mask[:, 0], attention_mask[:, 1])).to(device)
    labels = labels.repeat(2).to(device)

    # Data augmentation handled by scibert, dropout implemented under the hood
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    embeddings = outputs.last_hidden_state[:, 0]
    return embeddings, labels

In [ ]:
miner = miners.MultiSimilarityMiner()
loss_func = losses.NTXentLoss(temperature=0.07)

def train_and_save(save_directory, train_dataloader, mining=False, model_name='allenai/scibert_scivocab_uncased'):
    model = AutoModel.from_pretrained(model_name).to(device)
    model.train()

    optimizer = AdamW(model.parameters(), lr=2e-5)
    epochs = 2
    num_training_steps = len(train_dataloader) * epochs
    num_warmup_steps = math.ceil(num_training_steps * 0.1)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)

    for epoch in range(epochs):
        total_loss = 0
        # Shape = [#features, #batch_size, #tensor_length]
        start = time.time()
        for i, batch in enumerate(train_dataloader):
            optimizer.zero_grad()

            embeddings, labels = encoder(batch, model)
            if mining:
                hard_pairs = miner(embeddings, labels)
                loss = loss_func(embeddings, labels, hard_pairs)
            else:
                loss = loss_func(embeddings, labels)

            loss.backward()
            optimizer.step()
            scheduler.step()

            total_loss += loss.item()

            if i % 1000 == 0:
                print(f"Batch: {i+1}/{len(train_dataloader)}")

        print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_dataloader)}, Time Taken: {(time.time()-start)/60}")

    model.save_pretrained(save_directory)

In [ ]:
train_dataloader = CitationDataSet('for_gcolab/scicite_nodup_section.csv').get_dataloader()
#train_and_save('for_gcolab/section_with_hard', train_dataloader, True) # no warmup steps
train_and_save('for_gcolab/scicite_section_without_hard_warmup', train_dataloader, False)


Batch: 1/3868
Batch: 1001/3868
Batch: 2001/3868
Batch: 3001/3868
Epoch 1, Loss: 2.2190360916622422, Time Taken: 41.057815817991894
Batch: 1/3868
Batch: 1001/3868
Batch: 2001/3868
Batch: 3001/3868
Epoch 2, Loss: 2.000316165549432, Time Taken: 41.04714049100876


In [5]:
import math
import random
import numpy as np
import json
import torch
from torch import nn
from collections import defaultdict
from torch.utils.data import DataLoader

from transformers import AutoTokenizer, AutoModel
from datasets import load_dataset
from torch.optim import AdamW

from sklearn.metrics import f1_score

In [6]:
file_path_train = 'for_gcolab/train.jsonl'
file_path_dev = 'for_gcolab/dev.jsonl'
file_path_test = 'for_gcolab/test.jsonl'
train_data = []
dev_data = []
test_data = []
with open(file_path_train, 'r', encoding='utf-8') as file:
    for line in file:
        train_data.append(json.loads(line))
with open(file_path_dev, 'r', encoding='utf-8') as file:
    for line in file:
        dev_data.append(json.loads(line))
with open(file_path_test, 'r', encoding='utf-8') as file:
    for line in file:
        test_data.append(json.loads(line))

In [7]:
class CitationsDatasetWithoutInputExample():
    label_to_id = {'background': 0, 'method': 1, 'result': 2}
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, item):
        return self.data[item]['string'], CitationsDatasetWithoutInputExample.label_to_id[self.data[item]['label']]

In [8]:
batch_size = 16
train_dataset = CitationsDatasetWithoutInputExample(train_data)
train_batch_size = batch_size
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)

In [9]:
dev_dataset = CitationsDatasetWithoutInputExample(dev_data)
dev_batch_size = batch_size
dev_dataloader = DataLoader(dev_dataset, shuffle=False, batch_size=dev_batch_size)

In [10]:
class CitationIntentClassifier(nn.Module):
    def __init__(self, model_path, num_labels):
        super(CitationIntentClassifier, self).__init__()
        self.tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
        self.sentence_transformer = AutoModel.from_pretrained(model_path)
        self.classifier = nn.Linear(768, num_labels)

    def forward(self, input_texts):
        tokenised = self.tokenizer(input_texts, return_tensors='pt', truncation=True, padding='max_length', max_length=256)
        tokenised = tokenised.to(device)
        embeddings = self.sentence_transformer(**tokenised)
        cls_representation = embeddings.last_hidden_state[:, 0]
        return self.classifier(cls_representation)

def train_epoch(model, dataloader, loss_func, optimizer):
    model.train()
    total_loss = 0
    for input_texts, labels in dataloader:
        labels = labels.to(device)
        optimizer.zero_grad()
        output = model(input_texts)
        loss = loss_func(output, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Training loss: {total_loss / len(dataloader)}")

def evaluate(model, dataloader, loss_func):
    model.eval()
    total_loss = 0
    total_correct = 0
    with torch.no_grad():
        for input_texts, labels in dataloader:
            labels = labels.to(device)
            output = model(input_texts)
            loss = loss_func(output, labels)
            total_loss += loss.item()
            total_correct += (output.argmax(1) == labels).sum().item()

    print(f"Evaluation loss: {total_loss / len(dataloader)}")
    print(f"Evaluation accuracy: {total_correct / len(dataloader.dataset)}")

In [11]:
test_dataset = CitationsDatasetWithoutInputExample(test_data)
test_batch_size = batch_size
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=test_batch_size)

In [12]:
def test(model, dataloader):
    model.eval()
    predictions = []
    true_labels = []

    with torch.no_grad():
        for input_texts, labels in dataloader:
            labels = labels.to(device)
            output = model(input_texts)
            _, predicted_labels = torch.max(output, dim=1)
            predictions.extend(predicted_labels.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
    return predictions, true_labels


def train_test_loop(model_name, num_epochs=5, learning_rate=2e-5):
    num_labels = 3
    citation_intent_classifier = CitationIntentClassifier(model_name, num_labels).to(device)

    optimizer = torch.optim.Adam(citation_intent_classifier.parameters(), lr=learning_rate)
    loss_func = torch.nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        train_epoch(citation_intent_classifier, train_dataloader, loss_func, optimizer)
        evaluate(citation_intent_classifier, dev_dataloader, loss_func)

    predictions, true_labels = test(citation_intent_classifier, test_dataloader)
    f1 = f1_score(true_labels, predictions, average='macro')
    print(f"F1 Score for {model_name}: {f1}")

In [ ]:
train_test_loop('allenai/scibert_scivocab_uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

Epoch 1/5
Training loss: 0.4096578717260629
Evaluation loss: 0.42965070736305466
Evaluation accuracy: 0.8449781659388647
Epoch 2/5
Training loss: 0.2614443693959782
Evaluation loss: 0.43242182980837496
Evaluation accuracy: 0.8406113537117904
Epoch 3/5
Training loss: 0.14280616639492413
Evaluation loss: 0.5636075658659483
Evaluation accuracy: 0.834061135371179
Epoch 4/5
Training loss: 0.06569968351348422
Evaluation loss: 0.7417600516165639
Evaluation accuracy: 0.8384279475982532
Epoch 5/5
Training loss: 0.0426638760390085
Evaluation loss: 0.7984709703074447
Evaluation accuracy: 0.8307860262008734
F1 Score: 0.852946306743048


In [ ]:
# Fine-BatchSize: 32, Pre-LR: 5e-5
train_test_loop('for_gcolab/section_with_hard')

Epoch 1/5
Training loss: 0.5484766691576602
Evaluation loss: 0.4548608301014736
Evaluation accuracy: 0.8329694323144105
Epoch 2/5
Training loss: 0.34377770642786065
Evaluation loss: 0.4483630610951062
Evaluation accuracy: 0.8460698689956332
Epoch 3/5
Training loss: 0.23769038635515427
Evaluation loss: 0.49048513893423407
Evaluation accuracy: 0.8406113537117904
Epoch 4/5
Training loss: 0.1534410563275911
Evaluation loss: 0.5846135433891724
Evaluation accuracy: 0.8384279475982532
Epoch 5/5
Training loss: 0.08914406673419614
Evaluation loss: 0.687878583011956
Evaluation accuracy: 0.8296943231441049
F1 Score: 0.8162177211044165


In [ ]:
# Fine-BatchSize: 32, Pre-LR: 2e-5
train_test_loop('for_gcolab/section_without_hard')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Epoch 1/5
Training loss: 0.4523975432727688
Evaluation loss: 0.4239066243171692
Evaluation accuracy: 0.8329694323144105
Epoch 2/5
Training loss: 0.2531581413318483
Evaluation loss: 0.46768848443853445
Evaluation accuracy: 0.8395196506550219
Epoch 3/5
Training loss: 0.126963294098558
Evaluation loss: 0.5451935015875717
Evaluation accuracy: 0.834061135371179
Epoch 4/5
Training loss: 0.055823071306578584
Evaluation loss: 0.6518451201504675
Evaluation accuracy: 0.8471615720524017
Epoch 5/5
Training loss: 0.031220848735714287
Evaluation loss: 0.8292831377736454
Evaluation accuracy: 0.8362445414847162
F1 Score: 0.8357675287569054


In [ ]:
# Fine-BatchSize: 16, Pre-LR: 2e-5, 4 epochs
train_test_loop('for_gcolab/section_without_hard')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Epoch 1/5
Training loss: 0.43868237657313663
Evaluation loss: 0.42730324381384355
Evaluation accuracy: 0.8384279475982532
Epoch 2/5
Training loss: 0.2420807163584833
Evaluation loss: 0.4624652760424491
Evaluation accuracy: 0.8548034934497817
Epoch 3/5
Training loss: 0.10921322188859936
Evaluation loss: 0.6053152700629214
Evaluation accuracy: 0.8449781659388647
Epoch 4/5
Training loss: 0.0529089880539348
Evaluation loss: 0.7270640227140794
Evaluation accuracy: 0.8406113537117904
Epoch 5/5
Training loss: 0.036261926282794205
Evaluation loss: 0.91127725550102
Evaluation accuracy: 0.8460698689956332
F1 Score: 0.840044137165776


In [ ]:
# Fine-BatchSize: 16, Pre-LR: 2e-5, 4 epochs
train_test_loop('for_gcolab/section_without_hard_warmup')

Epoch 1/5
Training loss: 0.4233064498145913
Evaluation loss: 0.4304370592380392
Evaluation accuracy: 0.8395196506550219
Epoch 2/5
Training loss: 0.24185827332584894
Evaluation loss: 0.47118923736029655
Evaluation accuracy: 0.8395196506550219
Epoch 3/5
Training loss: 0.11935387288000031
Evaluation loss: 0.6031839823799914
Evaluation accuracy: 0.8362445414847162
Epoch 4/5
Training loss: 0.05759026855140087
Evaluation loss: 0.6985285966817675
Evaluation accuracy: 0.8329694323144105
Epoch 5/5
Training loss: 0.039147372554110944
Evaluation loss: 0.8427344234981413
Evaluation accuracy: 0.8395196506550219
F1 Score for for_gcolab/section_without_hard_warmup: 0.8420409052746064


In [ ]:
# Fine-BatchSize: 16, Pre-LR: 2e-5, 4 epochs
train_test_loop('for_gcolab/section_without_hard_warmup_2epoch')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

Epoch 1/5
Training loss: 0.41384257838967464
Evaluation loss: 0.41382220798525315
Evaluation accuracy: 0.8537117903930131
Epoch 2/5
Training loss: 0.24409778983974756
Evaluation loss: 0.44852032910647066
Evaluation accuracy: 0.851528384279476
Epoch 3/5
Training loss: 0.12740369594941103
Evaluation loss: 0.5723868770332172
Evaluation accuracy: 0.8417030567685589
Epoch 4/5
Training loss: 0.05987485089670287
Evaluation loss: 0.7946388977258627
Evaluation accuracy: 0.834061135371179
Epoch 5/5
Training loss: 0.037455960778646304
Evaluation loss: 0.8821846289833173
Evaluation accuracy: 0.8482532751091703
F1 Score for for_gcolab/section_without_hard_warmup_2epoch: 0.8457944756861631


In [14]:
# Fine-BatchSize: 16, Pre-LR: 2e-5, 4 epochs
train_test_loop('for_gcolab/scicite_nodup_section_without_hard_warmup_2epoch')

Epoch 1/5
Training loss: 0.41318123559503594
Evaluation loss: 0.4203477072818526
Evaluation accuracy: 0.8537117903930131
Epoch 2/5
Training loss: 0.25415029966041386
Evaluation loss: 0.47177086770534515
Evaluation accuracy: 0.8351528384279476
Epoch 3/5
Training loss: 0.1343837882303669
Evaluation loss: 0.6279653115840308
Evaluation accuracy: 0.8417030567685589
Epoch 4/5
Training loss: 0.07060656344040672
Evaluation loss: 0.7080998621431405
Evaluation accuracy: 0.8362445414847162
Epoch 5/5
Training loss: 0.045066840957278066
Evaluation loss: 0.782965902110626
Evaluation accuracy: 0.8231441048034934
F1 Score for for_gcolab/scicite_nodup_section_without_hard_warmup_2epoch: 0.8281101800153098
